In [36]:
import pandas as pd
import torch
import numpy as np
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_experimental.text_splitter import SemanticChunker
import torch
from gliner import GLiNER
from langchain_experimental.text_splitter import SemanticChunker
import re
from pprint import pprint
import random
from utilities import *
from chunking import EnhancedSemanticChunker
from langchain_qdrant import Qdrant
from initialize_groq import init_groq
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
import pprint

In [37]:
df = pd.read_csv("data/emails.csv")
idx = 1000
msg = df['message'][idx]
VECTOR_DB_NAME = "emails_e5_qdrant"

THIS IS TESTING

In [38]:
print(msg)
print(clean_text(msg))
print('cuda available?',torch.cuda.is_available())

Message-ID: <1081797.1075855696183.JavaMail.evans@thyme>
Date: Thu, 15 Mar 2001 07:39:00 -0800 (PST)
From: phillip.allen@enron.com
To: stagecoachmama@hotmail.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: stagecoachmama@hotmail.com
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_June2001\Notes Folders\All documents
X-Origin: Allen-P
X-FileName: pallen.nsf

Lucy,

Here is the rentroll.

My only questions are about #18, #25, and #37 missed rent.  Any special 
reasons?

It looks like there are five vacancies #2,12,20a,35,40.  If you want to run 
an ad in the paper with a $50 discount that is fine.
I will write you a letter of recommendation.  When do you need it?  You can 
use me as a reference.  In the next two weeks we should really have a good 
idea whether the sale is going through.

Phillip
essage: 1081797.1075855696183.avaail.evans@thymeate: hu, 15 ar 2001 07:39:00 0800 ()rom: phillip.allen@enron.como:

In [39]:
# initializing Microsoft E5 model
model_kwargs = {'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
encode_kwargs = {'normalize_embeddings': True}  # does l2 norm for the cos sim
modelemb = HuggingFaceEmbeddings(
    model_name="intfloat/e5-base-v2", 
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

In [40]:
# initializing GLiNER model
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gliner_model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")
gliner_model.config.max_len = 512
gliner_model.to(DEVICE)

# model entity labels configuration
labels = ["date", "location", "person", "action", "finance", "legal", "event", "product", "organization"]

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 57456.22it/s]
/opt/homebrew/Caskroom/miniforge/base/envs/gditml/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [41]:
# initializing EnhancedSemanticChunker
enhanced_chunker = EnhancedSemanticChunker(
    embeddings=modelemb,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=50,  
    min_chunk_size=5,
    overlap_sentences=2,  
    gliner_model=gliner_model
)

In [42]:
metadata, split_msg = extract_email_metadata(msg, idx)
msg_start = split_msg.index("X-FileName:")
full_content = clean_text(" ".join(split_msg[msg_start + 2:]))
# print(full_content)
# Create document with the enhanced chunker
documents = enhanced_chunker.create_documents(
    texts=[full_content],
    metadatas=[metadata]
)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


THIS IS TESTING

In [43]:
print(f"Number of chunks after deduplication: {len(documents)}")

# Now use 'deduplicated_documents' for further processing (indexing, etc.)
for i, doc in enumerate(documents):  
    print(f"\n--- Unique Chunk {i+1}/{len(documents)} ---")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")

Number of chunks after deduplication: 5

--- Unique Chunk 1/5 ---
Content: passage: ucy, ere is the rentroll. y only questions are about 18, 25, and 37 missed rent. ny special reasons? t looks like there are five vacancies 2,12,20a,35,40. f you want to run an ad in the paper with a 50 discount that is fine.
Metadata: {'Message-ID': '<1081797.1075855696183.JavaMail.evans@thyme>', 'filename': 'allen-p/all_documents/462.', 'sender': 'phillip.allen@enron.com', 'recipient': 'stagecoachmama@hotmail.com', 'date': '03-15-2001', 'subject': '', 'entities': 'This passage contains locations mentioned include ny. '}

--- Unique Chunk 2/5 ---
Content: passage: y only questions are about 18, 25, and 37 missed rent. ny special reasons? t looks like there are five vacancies 2,12,20a,35,40. f you want to run an ad in the paper with a 50 discount that is fine. will write you a letter of recommendation. hen do you need it?
Metadata: {'Message-ID': '<1081797.1075855696183.JavaMail.evans@thyme>', 'filename'

In [44]:
db = Qdrant.from_existing_collection(modelemb, "emails_e5_qdrant", VECTOR_DB_NAME)
model = modelemb

In [ ]:
prompt = ChatPromptTemplate.from_template(
        """
            You are an expert in vector-based retrieval for Qdrant databases. The database stores semantically chunked email passages, each consisting of one sentence plus one sentence before and after for context. Each passage is enriched with metadata, including fields such as: date, location, person, action, finance, legal, event, product, and organization.
            Your task is to reformulate the following user question into a precise, high-quality natural language query optimized for semantic similarity search. Leverage the structure of the vector schema and, when appropriate, integrate terminology related to the metadata fields to improve retrieval relevance.
            Context: "{context}"
            Original user question: "{input}"
            Return ONLY the reformulated query, suitable for vector search. Do NOT include any explanations or additional text.
        """
)

document_prompt = PromptTemplate.from_template(
    "METADATA: Source: {sender}\nDate: {date}\n Recipients: {recipient}\nSubject: {subject}\nEntities: {entities}\n\nContent: {page_content}"
)

retriever = db.as_retriever(search_kwargs={'k':20, 'search_type':'mmr','lambda_mult':0.2})

_, llm, groqllm = init_groq(model_name="llama-3.3-70b-versatile")
document_chain = create_stuff_documents_chain(llm, prompt=prompt, document_prompt=document_prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Retrieve Top-K Similar Documents (Initial Broad Search)
# retriever_topk = db.as_retriever(search_kwargs={'k': 20,'fetch_k' : 100, 'search_type': 'similarity_s core_threshold','score_threshold':0.75})  # Retrieve more docs first
retriever_topk = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={'score_threshold':0.65,'k':10})
# MMR for Diversity (Reduce Redundant Docs)
retriever_mmr = db.as_retriever(search_type="mmr", search_kwargs={'k':10,'lambda_mult': 1})  

# Create the Hybrid Retrieval Pipeline
retrieval_chain_topk = create_retrieval_chain(retriever_topk, document_chain)  # Initial broad search
retrieval_chain_mmr = create_retrieval_chain(retriever_mmr, document_chain)    # Apply MMR re-ranking

THIS IS TESTING

In [46]:
query = "query: is MSEB an indian company? 402 crore amount? its relation to enron?"
# pprint.pprint(retrieval_chain_topk.invoke({"input":query}))
pprint.pprint(retrieval_chain_mmr.invoke({"input":query}))

test_questions = [
    "What does randy need to send a schedule of?",
    "What are some of randy's action items?",
    "What is Philip's proposal focused on, and can you provided details about the proposal?",
    "Can you provide me more detail about the microturbine power generation deal?",
    "What needs to be faxed?",
    "Are there hints of a scandal in the emails?",
    "What did jeffrey skilling tell john arnold"
]
for text in test_questions:
    # Define query
    query = "query: " + text
    pprint.pprint(retrieval_chain_mmr.invoke({"input":query}))

test_questions = [
    "query: What does randy need to send a schedule of?",
    "query: What are some of randy's action items?",
    "query: What is Philip's proposal focused on, and can you provided details about the proposal?",
    "query: Can you provide me more detail about the microturbine power generation deal?",
    "query: What needs to be faxed?"
]
for text in test_questions:
    print("=========================================================")
    query = "query: " + text
    query_embedding = np.array(model.embed_query(query))
    # query_embedding = l2_normalize(query_embedding)  
    topk_results = db.similarity_search_with_score_by_vector(
        embedding=query_embedding.tolist(),  # List[float]
        k=5
    )

    mmr_results = db.max_marginal_relevance_search_with_score_by_vector(
        embedding=query_embedding.tolist(),  # List[float]
        k=5,
        lambda_mult=0.8         
    )

    # Sort by L2 distance (ascending: lower = more similar)
    topk_sorted = sorted(topk_results, key=lambda x: x[1])
    
    mmr_sorted = sorted(mmr_results, key=lambda x: x[1], reverse=True)

    # Display results with L2 distance and cosine similarity
    for doc, mmr_score in mmr_sorted:
        # docembedding = l2_normalize(np.array(modelemb.embed_documents([doc.page_content])))
        # cos_sim = float(np.dot(query_embedding, docembedding.reshape(-1)))
        pprint.pprint(f"Document:\n {doc.page_content} | MMR Score: {mmr_score:.4f}")
        
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    for doc, l2_score in topk_sorted:
        # Convert L2 distance to cosine similarity (assuming L2-normalized)
        cosine_sim = 1 - (l2_score ** 2) / 2
        pprint.pprint(f"Document: {doc.page_content[:100]} | L2 Distance: {l2_score:.4f} | Cosine Sim: {cosine_sim:.4f}")

ValueError: Collection emails_e5_qdrant not found

In [ ]:
def call_model(state: MessagesState):
    state["messages"]
    messages = state["messages"]
    #print(messages)
    groqllm.groq_api_key = random.choice(api_keys)
    llm_with_tool = groqllm.bind_tools([ragtool])
    response = llm_with_tool.invoke(messages)
    
    
    return {"messages": [response]}

@tool
def ragtool(query: str, num_docs: int) -> str:
    """
    This is a retrieval-augmented generation (RAG) tool that queries a vector store 
    containing Enron emails.
    
    Parameters:
    query (str): The input query for retrieval.
    num_docs (int): The number of documents to retrieve.
    Returns:
    str: The retrieved answer from the vector store.
    """
    try:
        answer = retrieval_chain_topk.invoke({"input": query})['answer']
        return f"Here is the ANSWER. \n ```{answer}```\n DO NOT USE THE TOOL REPEATEDLY. SHOW THE ANSWER TO THE USER. \n"
    except Exception as e:
        return f"Error: Failed to retrieve answer. Details: {str(e)}"

In [ ]:
toolnode = ToolNode([ragtool])

memory = MemorySaver()
workflow = StateGraph(MessagesState)    
workflow.add_node("agent", call_model)
workflow.add_node(toolnode)
workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent",
    router_function,
    {
       "tools": "tools",
       END: END,
    },
)
workflow.add_edge("tools", "agent")
app = workflow.compile(checkpointer=memory)

THIS IS TESTING

In [ ]:
from IPython.display import display_png
display_png(app.get_graph().draw_mermaid_png(),raw=True)

import time
while True:
    theinput = input("Enter something: ")
    if 'exit' in theinput:
        break
    inp = {"messages":[theinput]}
    
    config = {"configurable": {"thread_id": 1}}
    events = app.stream(inp, config=config, stream_mode="values")

    for event in events:
        event["messages"][-1].pretty_print()
    time.sleep(1)

In [ ]:
# Define the multi-query prompt template
# This template instructs the LLM to generate multiple search queries from a single user question
multi_template = """You are an expert at querying search engines. You specialize in understanding natural language queries and generating multiple search
queries that, taken together, would help provide a comprehensive answer to the user's question.

Main Question: {question}

Let's break this down. Generate 4 search queries for querying a knowledge store about emails. 
Make sure these queries use language that would appear in actual emails.
Remember to keep them short, using keywords that would be found in emails.
Keep them straightforward and distinct from each other.
Formulate them from different angles to solve the main query.

Return a bullet list with • at the start of each question:

• query 1
• query 2
• etc.
"""

# Create a processor to generate multiple search queries from a single question
multi_query_prompt = PromptTemplate.from_template(multi_template)
multi_query_chain = multi_query_prompt | llm | StrOutputParser()

In [ ]:
def run_multi_query(main_query, query_generator=multi_query_chain, single_query_chain=retrieval_chain_topk):
    """
    Run a multi-query retrieval process to improve search results accuracy.
    
    This function:
    1. Takes a user query and generates multiple search queries using the LLM
    2. Executes each generated query against the retrieval system
    3. Combines and summarizes the results for a comprehensive answer
    
    Args:
        main_query: The original user question
        query_generator: Chain to generate multiple search queries
        single_query_chain: Chain to execute individual queries
        
    Returns:
        Dictionary with consolidated results and performance metrics
    """
    # Start timing the process
    start_time = total_start_time = time.time()
    
    # Generate multiple search queries from the main question
    result = query_generator.invoke({"question": main_query})
    
    # Extract the generated queries from the bullet point list
    sub_questions = [q.strip() for q in result.split('•') if q.strip()]
    
    # Record query generation time
    gen_time = time.time() - start_time
    start_time = time.time()
    
    # Track all retrieved documents and their sources
    all_docs = []
    all_results = []
    
    # Process each generated query
    print("Generated Questions:")
    for i, question in enumerate(sub_questions):
        print(f"{i+1}. {question}")
        # Execute the query against the retrieval system
        chain_result = single_query_chain.invoke({"input": question})
        all_results.append(chain_result)
        
        # Track the documents retrieved for this query
        if "context" in chain_result:
            all_docs.extend(chain_result["context"])
    
    # Record search execution time
    search_time = time.time() - start_time
    
    # Calculate combined result using the original query
    # This ensures the answer is based on all retrieved information
    final_answer = single_query_chain.invoke({
        "input": main_query,
        "context": all_docs[:10]  # Limit to top 10 most relevant documents
    })
    
    # Record total processing time
    total_time = time.time() - total_start_time
    
    # Return comprehensive results with timing metrics
    return {
        "main_query": main_query,
        "generated_queries": sub_questions,
        "individual_answers": all_results,
        "final_answer": final_answer["answer"],
        "all_docs": all_docs,
        "timing": {
            "query_generation": gen_time,
            "search_execution": search_time,
            "total_processing": total_time
        }
    }

In [ ]:
# Define test queries for evaluating retrieval performance
test_queries = [
    "What do we know about Skilling's involvement in Enron's financial reporting?",
    "What are the main topics discussed in emails from Kenneth Lay?",
    "How did Enron executives discuss the California energy crisis in their emails?",
    "What discussions were happening about LJM partnerships in the months before Enron's collapse?",
    "What was discussed about mark-to-market accounting in emails?",
    "Who was responsible for overseeing Special Purpose Entities at Enron?",
    "What communication happened regarding Raptor structures?",
]

In [ ]:
# Example usage - process a test query with advanced retrieval
for i in range(len(test_queries)):
    result = run_multi_query(test_queries[i])
    # Print the final answer
    print(result["final_answer"])